#  Mission to Mars ETL Project

## Import Required Modules 

In [1]:
#Import our Modules
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import time 

## URL Variables

In [2]:
site_base = 'https://astrogeology.usgs.gov'
mars_latest_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
mars_gallery_url = 'https://www.jpl.nasa.gov/images?search='
mars_facts_url = 'https://space-facts.com/mars/'
mars_hemisphers_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced'

## Get Latest News

In [3]:
html = requests.get(mars_latest_news_url)
bs = BeautifulSoup(html.text,'html.parser')

In [4]:
results = bs.find("div",class_="slide")
paragraph = results.a.text.strip()
title = results.find('div',class_="content_title").text.strip()


In [5]:
print(paragraph)
print(title)

Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.
NASA Ingenuity Mars Helicopter Prepares for First Flight


## Splinter Work

In [6]:
# Setup splinter browser driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(mars_gallery_url)
time.sleep(2)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\mreg9\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [7]:
browser.find_by_css("input[id=filter_Mars]").first.click()   


In [8]:
time.sleep(2)
bs = BeautifulSoup(browser.html,'html.parser')
results = bs.find("div",class_="SearchResultCard")
print(results)

<div class="SearchResultCard" page_id="127961" page_score="1"><a class="group cursor-pointer block" href="/images/dawes-crater-false-color"><div class="border-gray-light-mid lg:pb-10 lg:mb-10 relative grid grid-cols-9 gap-5 pb-6 mb-6 border-b"><div class="lg:hidden col-span-full text-subtitle"><span>
        Image
        <span class="sr-only">.</span></span> <!-- --> <!-- --></div> <div class="sm:col-span-6 md:col-span-6 lg:col-span-7 xl:col-span-7 col-span-6"><div class="text-subtitle lg:block hidden mb-3"><span>
          Image
          <span class="sr-only">.</span></span> <!-- --> <!-- --></div> <h2 class="mb-3 text-h5">Dawes Crater - False Color</h2> <!-- --> <p class="text-gray-mid-dark">Mar 26, 2021</p></div> <div class="SearchResultCard-image xl:pl-14 sm:col-span-3 md:col-span-3 lg:col-span-2 xl:col-span-2 col-span-3"><div data-v-a6031820=""><div class="BaseImagePlaceholder aspect-ratio-one-one dark-theme" data-v-a6031820=""><div class="sm:object-cover object-cover" data-v-a6

In [9]:
large_image_html = results.a['href']

In [10]:
browser.click_link_by_href(large_image_html)

D:\ProgramData\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:474: FutureWarning: browser.find_link_by_href is deprecated. Use browser.links.find_by_href instead.
  warnings.warn(


In [11]:
bs = BeautifulSoup(browser.html,'html.parser')
results = bs.find("div",class_= "BaseImagePlaceholder")

In [12]:
featured_image_url = results.img['src']

In [13]:
print(featured_image_url)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24510.2e16d0ba.fill-400x400-c50.jpg


##  Mars Facts Table Scraping With Pandas

In [14]:
html = requests.get(mars_facts_url)

In [15]:
df = pd.read_html(html.text)
mars_facts_df = df[0]

cols = ["Stats", "Values"]

mars_facts_df.set_axis(cols,axis="columns",inplace=True)
mars_facts_df.set_index(["Stats","Values"],inplace=True)

In [16]:
mars_facts_html_table = mars_facts_df.to_html()


In [17]:
mars_facts_html_table = mars_facts_html_table.replace('\n', '')

In [18]:
browser.visit(mars_hemisphers_url)

In [19]:
bs = BeautifulSoup(browser.html,'html.parser')

In [20]:
results = bs.find_all('div', class_="item")

In [21]:
hemi_image_url_list = []
for result in results:
    hemi_dict = {}
    link = result.a['href'].strip()
    title_url = result.h3.text
    browser.visit(site_base + link)
    bs = BeautifulSoup(browser.html,'html.parser')
    res = bs.find('img',class_='wide-image')
    img_url = site_base + res['src']
    hemi_dict = {"title":title_url, "img_url":img_url}
    hemi_image_url_list.append(hemi_dict)


In [22]:
print(hemi_image_url_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


## Create Data Dictionary of all scraped data 

In [25]:
final_dict = {
    "LatestNewsTitle": title,
    "LatestNewsParagraph":paragraph,
    "FeatureImageUrl": featured_image_url,
    "MarsFactHtml":mars_facts_html_table,
    "HemispherImageUrlList":hemi_image_url_list
}

In [26]:
final_dict

{'LatestNewsTitle': 'NASA Ingenuity Mars Helicopter Prepares for First Flight',
 'LatestNewsParagraph': 'Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.',
 'FeatureImageUrl': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24510.2e16d0ba.fill-400x400-c50.jpg',
 'MarsFactHtml': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>    </tr>    <tr>      <th>Stats</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <th>6,792 km</th>    </tr>    <tr>      <th>Polar Diameter:</th>      <th>6,752 km</th>    </tr>    <tr>      <th>Mass:</th>      <th>6.39 × 10^23 kg (0.11 Earths)</th>    </tr>    <tr>      <th>Moons:</th>      <th>2 (Phobos &amp; Deimos)</th>    </tr>    <tr>      <th>Orbit Distance:</th>      <th>227,943,824 km (1.38 AU)</th>    </tr>    <tr>      <th>Orbit Period:</th>      <th>687 days (1.9 years)</th> 

In [27]:
final_dict2 = {
    "LatestNewsTitle": title,
    "LatestNewsParagraph":paragraph,
    "FeatureImageUrl": featured_image_url,
    "MarsFactHtml":mars_facts_html_table
}

In [ ]:
for hemi in HemispherImageUrlList:
    final_dict2[hemi.title] = 